# 🧪 Invoking APIs with `OpenAPITool`

<a href="https://colab.research.google.com/github/deepset-ai/haystack-experimental/blob/main/examples/openapitool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" width="200"/></a>

Many APIs available on the Web provide an OpenAPI specification that describes their structure and syntax.

[`OpenAPITool`](https://docs.haystack.deepset.ai/reference/openapi-api) is an experimental Haystack component that allows you to call an API using payloads generated from human instructions.

Here's a brief overview of how it works:
- At initialization, it loads the OpenAPI specification from a URL or a file.
- At runtime:
  - Converts human instructions into a suitable API payload using a Chat Language Model (LLM).
  - Invokes the API.
  - Returns the API response, wrapped in a Chat Message.

Let's see this component in action...

## Setup

In [ ]:
! pip install haystack-ai jsonref

In this notebook, we will be using some APIs that require an API key. Let's set them as environment variables.

In [ ]:
import os

os.environ["OPENAI_API_KEY"]="..."

# free API key: https://www.firecrawl.dev/
os.environ["FIRECRAWL_API_KEY"]="..."

# free API key: https://serper.dev/
os.environ["SERPERDEV_API_KEY"]="..."

## Call an API without credentials

In the first example, we use Open-Meteo, a Free Weather API that does not require authentication.

We use `OPENAI` as LLM provider. Other supported providers are `ANTHROPIC` and `COHERE`.

In [ ]:
from haystack.dataclasses import ChatMessage
from haystack_experimental.components.tools.openapi import OpenAPITool, LLMProvider
from haystack.utils import Secret

tool = OpenAPITool(generator_api=LLMProvider.OPENAI,
                   spec="https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml")

tool.run(messages=[ChatMessage.from_user("Weather in San Francisco, US")])

{'service_response': [ChatMessage(content='{"latitude": 37.763283, "longitude": -122.41286, "generationtime_ms": 0.07903575897216797, "utc_offset_seconds": 0, "timezone": "GMT", "timezone_abbreviation": "GMT", "elevation": 18.0, "current_weather_units": {"time": "iso8601", "interval": "seconds", "temperature": "\\u00b0C", "windspeed": "km/h", "winddirection": "\\u00b0", "is_day": "", "weathercode": "wmo code"}, "current_weather": {"time": "2024-07-22T16:45", "interval": 900, "temperature": 18.1, "windspeed": 3.4, "winddirection": 212, "is_day": 1, "weathercode": 1}}', role=<ChatRole.USER: 'user'>, name=None, meta={})]}

## Incorporate `OpenAPITool` in a Pipeline

Next, let's create a simple Pipeline where the service response is translated into a human-understandable format using the Language Model.

We use an [`OutputAdapter`](https://docs.haystack.deepset.ai/docs/outputadapter) to create a list of Chat Messages for the LM.

In [ ]:
from typing import List
from haystack import Pipeline
from haystack.components.converters import OutputAdapter
from haystack.components.generators.chat import OpenAIChatGenerator

pipe = Pipeline()
pipe.add_component("meteo", tool)
pipe.add_component("prompt_adapter", OutputAdapter("{{user_message + service_response}}", List[ChatMessage]))
pipe.add_component("llm", OpenAIChatGenerator(generation_kwargs={"max_tokens": 1024}))

pipe.connect("meteo", "prompt_adapter.service_response")
pipe.connect("prompt_adapter", "llm.messages")

🚅 Components
  - meteo: OpenAPITool
  - prompt_adapter: OutputAdapter
  - llm: OpenAIChatGenerator
🛤️ Connections
  - meteo.service_response -> prompt_adapter.service_response (List[ChatMessage])
  - prompt_adapter.output -> llm.messages (List[ChatMessage])

In [ ]:
result = pipe.run(data={"meteo": {"messages": [ChatMessage.from_user("weather in San Francisco, US")]},
                        "prompt_adapter": {"user_message": [ChatMessage.from_user("Explain the weather in San Francisco in a human understandable way")]}})

In [ ]:
print(result["llm"]["replies"][0].content)

Currently in San Francisco, the weather can be described as mild with a temperature of 18.1°C (64.6°F). The wind is blowing at a speed of 3.4 km/h coming from the south-southwest direction. It is daytime in San Francisco at the moment.


## Use an API with credentials in a Pipeline

In this example, we use [Firecrawl](https://www.firecrawl.dev/): a project that scrape Web pages (and Web sites) and convert them into clean text. Firecrawl has an API that requires an API key.

In the following Pipeline, we use Firecrawl to scrape a news article, which is then summarized using a Language Model.

In [ ]:
pipe = Pipeline()
pipe.add_component("firecrawl", OpenAPITool(generator_api=LLMProvider.OPENAI,
                                            spec="https://raw.githubusercontent.com/mendableai/firecrawl/main/apps/api/openapi.json",
                                            credentials=Secret.from_env_var("FIRECRAWL_API_KEY")))
pipe.add_component("prompt_adapter", OutputAdapter("{{user_message + service_response}}", List[ChatMessage]))
pipe.add_component("llm", OpenAIChatGenerator(generation_kwargs={"max_tokens": 1024}))

pipe.connect("firecrawl", "prompt_adapter.service_response")
pipe.connect("prompt_adapter", "llm.messages")

🚅 Components
  - firecrawl: OpenAPITool
  - prompt_adapter: OutputAdapter
  - llm: OpenAIChatGenerator
🛤️ Connections
  - firecrawl.service_response -> prompt_adapter.service_response (List[ChatMessage])
  - prompt_adapter.output -> llm.messages (List[ChatMessage])

In [ ]:
user_prompt = "Given the article below, list the most important facts in a bulleted list. Do not include repetitions. Max 5 points."

result = pipe.run(data={"firecrawl": {"messages": [ChatMessage.from_user("Scrape https://lite.cnn.com/2024/07/18/style/rome-ancient-papal-palace/index.html")]},
                        "prompt_adapter": {"user_message": [ChatMessage.from_user(user_prompt)]}})

In [ ]:
print(result["llm"]["replies"][0].content)

- Remains of a medieval palace believed to be where popes lived before the Vatican have been excavated in Rome.
- The site is located near the Archbasilica of St John Lateran in Rome.
- The building's initial structure dates back to Emperor Constantine in the 4th century and was expanded between the 9th and 13th centuries.
- The papacy resided in the palace until 1305 when it temporarily moved to Avignon in France.
- The discovery was made ahead of renovations for the 2025 Catholic Holy Year, expected to attract over 30 million pilgrims and tourists to Rome.


## Create a Pipeline with multiple `OpenAPITool` components

In this example, we show a Pipeline where multiple alternative APIs can be invoked depending on the user query. In particular, a Google Search (via Serper.dev) can be performed or a single page can be scraped using Firecrawl.

⚠️ The approach shown is just one way to achieve this using [conditional routing](https://docs.haystack.deepset.ai/docs/conditionalrouter). We are currently experimenting with tool support in Haystack, and there may be simpler ways to achieve the same result in the future.

In [ ]:
import json

decision_prompt_template = """
You are a virtual assistant, equipped with the following tools:

- `{"tool_name": "search_web", "tool_description": "Access to Google search, use this tool whenever information on recents events is needed"}`
- `{"tool_name": "scrape_page", "tool_description": "Use this tool to scrape and crawl web pages"}`

Select the most appropriate tool to resolve the user's query. Respond in JSON format, specifying the user request and the chosen tool for the response.
If you can't match user query to an above listed tools, respond with `none`.


######
Here are some examples:

```json
{
  "query": "Why did Elon Musk recently sue OpenAI?",
  "response": "search_web"
}
{
  "query": "What is on the front-page of hackernews today?",
  "response": "scrape_page"
}
{
  "query": "Tell me about Berlin",
  "response": "none"
}
```

Choose the best tool (or none) for each user request, considering the current context of the conversation specified above.

{"query": {{query}}, "response": }
"""

def get_tool_name(replies):
  try:
    tool_name = json.loads(replies)["response"]
    return tool_name
  except:
    return "error"

def create_chat_message(query):
  return [ChatMessage.from_user(query)]


routes = [
    {
        "condition": "{{replies[0] | get_tool_name == 'search_web'}}",
        "output": "{{query | create_chat_message}}",
        "output_name": "search_web",
        "output_type": List[ChatMessage],
    },
    {
        "condition": "{{replies[0] | get_tool_name == 'scrape_page'}}",
        "output": "{{query | create_chat_message}}",
        "output_name": "scrape_page",
        "output_type": List[ChatMessage],
    },
    {
        "condition": "{{replies[0] | get_tool_name == 'none'}}",
        "output": "{{replies[0]}}",
        "output_name": "no_tools",
        "output_type": str,
    },
    {
        "condition": "{{replies[0] | get_tool_name == 'error'}}",
        "output": "{{replies[0]}}",
        "output_name": "error",
        "output_type": str,
    },
]

In [ ]:
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.routers import ConditionalRouter
from haystack.components.generators import OpenAIGenerator

search_web_tool = OpenAPITool(generator_api=LLMProvider.OPENAI,
                   spec="https://bit.ly/serper_dev_spec_yaml",
                   credentials=Secret.from_env_var("SERPERDEV_API_KEY"))

scrape_page_tool = OpenAPITool(generator_api=LLMProvider.OPENAI,
                   spec="https://raw.githubusercontent.com/mendableai/firecrawl/main/apps/api/openapi.json",
                   credentials=Secret.from_env_var("FIRECRAWL_API_KEY"))

pipe = Pipeline()
pipe.add_component("prompt_builder", PromptBuilder(template=decision_prompt_template))
pipe.add_component("llm", OpenAIGenerator())
pipe.add_component("router", ConditionalRouter(routes, custom_filters={"get_tool_name": get_tool_name,
                                                                       "create_chat_message": create_chat_message}))
pipe.add_component("search_web_tool", search_web_tool)
pipe.add_component("scrape_page_tool", scrape_page_tool)

pipe.connect("prompt_builder", "llm")
pipe.connect("llm.replies", "router.replies")
pipe.connect("router.search_web", "search_web_tool")
pipe.connect("router.scrape_page", "scrape_page_tool")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
  - router: ConditionalRouter
  - search_web_tool: OpenAPITool
  - scrape_page_tool: OpenAPITool
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)
  - llm.replies -> router.replies (List[str])
  - router.search_web -> search_web_tool.messages (List[ChatMessage])
  - router.scrape_page -> scrape_page_tool.messages (List[ChatMessage])

In [ ]:
query = "Who won the UEFA European Football Championship?"

pipe.run({"prompt_builder": {"query": query}, "router": {"query": query}})

{'llm': {'meta': [{'model': 'gpt-3.5-turbo-0125',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 23,
     'prompt_tokens': 248,
     'total_tokens': 271}}]},
 'search_web_tool': {'service_response': [ChatMessage(content='{"searchParameters": {"q": "UEFA European Football Championship winner", "type": "search", "engine": "google"}, "answerBox": {"title": "Spain national football teamUEFA European Championship / Latest Champion", "answer": "Spain national football team"}, "organic": [{"title": "UEFA European Championship - Wikipedia", "link": "https://en.wikipedia.org/wiki/UEFA_European_Championship", "snippet": "The most recent championship, held in Germany in 2024, was won by Spain, who lifted a record fourth European title after beating England 2\\u20131 in the final at the ...", "sitelinks": [{"title": "Finals", "link": "https://en.wikipedia.org/wiki/List_of_UEFA_European_Championship_finals"}, {"title": "European Championship in", "link": "https://en

In [ ]:
query = "What is on the front-page of BBC today?"

pipe.run({"prompt_builder": {"query": query}, "router": {"query": query}})

{'llm': {'meta': [{'model': 'gpt-3.5-turbo-0125',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 26,
     'prompt_tokens': 250,
     'total_tokens': 276}}]},
 'scrape_page_tool': {'service_response': [ChatMessage(content='{"success": true, "data": {"content": "\\n\\n[British Broadcasting Corporation](/)\\n\\n[Watch Live](/watch-live-news)\\n\\nRegisterSign In\\n\\n*   [Home](/)\\n    \\n*   [News](/news)\\n    \\n*   [Sport](/sport)\\n    \\n*   [Business](/business)\\n    \\n*   [Innovation](/innovation)\\n    \\n*   [Culture](/culture)\\n    \\n*   [Travel](/travel)\\n    \\n*   [Earth](/future-planet)\\n    \\n*   [Video](/video)\\n    \\n*   [Live](/live)\\n    \\n\\nRegisterSign In\\n\\n[Home](/)\\n\\nNews\\n\\n[Sport](/sport)\\n\\nBusiness\\n\\nInnovation\\n\\nCulture\\n\\nTravel\\n\\nEarth\\n\\n[Video](/video)\\n\\nLive\\n\\n[Audio](https://www.bbc.co.uk/sounds)\\n\\n[Weather](https://www.bbc.com/weather)\\n\\n[Newsletters](https://www.bbc.com/ne